In [6]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [7]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
credit_df = pd.read_csv("../content/lending_data.csv")

# Review the DataFrame
credit_df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [9]:
credit_df.columns

Index(['loan_size', 'interest_rate', 'borrower_income', 'debt_to_income',
       'num_of_accounts', 'derogatory_marks', 'total_debt', 'loan_status'],
      dtype='object')

In [10]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = credit_df["loan_status"]

# Separate the X variable, the features
x = credit_df[['loan_size', 'interest_rate', 'borrower_income', 'debt_to_income',
       'num_of_accounts', 'derogatory_marks', 'total_debt']]

In [15]:
# Review the y variable Series
print(y[:5])

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64


In [16]:
# Review the X variable DataFrame
print(x[:5])

   loan_size  interest_rate  borrower_income  debt_to_income  num_of_accounts  \
0    10700.0          7.672            52800        0.431818                5   
1     8400.0          6.692            43600        0.311927                3   
2     9000.0          6.963            46100        0.349241                3   
3    10700.0          7.664            52700        0.430740                5   
4    10800.0          7.698            53000        0.433962                5   

   derogatory_marks  total_debt  
0                 1       22800  
1                 0       13600  
2                 0       16100  
3                 1       22700  
4                 1       23000  


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [17]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [20]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1, stratify=y, train_size=0.7, test_size=0.3)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [26]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
model = LogisticRegression(solver='lbfgs', max_iter=200, random_state=1)

# Fit the model using training data
fitted_model = model.fit(x_train, y_train)
fitted_model

LogisticRegression(max_iter=200, random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [27]:
# Make a prediction using the testing data
x_test_pred_1 = fitted_model.predict(x_test)
x_test_credit_df = pd.DataFrame({
    "test" : y_test,
    "pred" : x_test_pred_1
})
x_test_credit_df.sample(10)

,test,pred
63768,0,0
17551,0,0
22243,0,0
30728,0,0
14355,0,0
13863,0,0
54888,0,0
52373,0,0
14136,0,0
62473,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [30]:
# Print the balanced_accuracy score of the model
print(balanced_accuracy_score(y_test, x_test_pred_1))

0.9537344409399849


In [31]:
# Generate a confusion matrix for the model
print(confusion_matrix(y_test, x_test_pred_1))

[[22409   102]
 [   66   684]]


In [33]:
# Print the classification report for the model
print(classification_report(y_test, x_test_pred_1))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     22511
           1       0.87      0.91      0.89       750

    accuracy                           0.99     23261
   macro avg       0.93      0.95      0.94     23261
weighted avg       0.99      0.99      0.99     23261



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** With an accuracy of 99%, the logistic regression model predicts the 0 and 1 labels well. The 0 labels had 100% accuracy in predictions, and the 1's had close to 90% average for the Precision, Recall and F1-Score categories.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points.

In [36]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
random_model = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
random_fitted_model = random_model.fit(x_train, y_train)
random_fitted_model

RandomOverSampler(random_state=1)

In [44]:
# Count the distinct values of the resampled labels data
print(len(y_train.unique()))

2


### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [58]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
model_2 = LogisticRegression(solver='lbfgs', max_iter=200, random_state=1)

# Fit the model using the resampled training data
x_train_resample, y_train_resample = random_fitted_model.fit_resample(x_train, y_train)

model_2.fit(x_train_resample, y_train_resample)

# Make a prediction using the testing data
x_test_pred_2 = model_2.predict(x_test)

x_test_random_credit_df = pd.DataFrame({
    "resampled test" : y_test,
    "resampled pred" : x_test_pred_2
})
x_test_random_credit_df.sample(10)

,resampled test,resampled pred
32379,0,0
3967,0,0
8542,0,0
11173,0,0
26136,0,0
68005,0,0
34885,0,0
65762,0,0
46956,0,0
32107,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [59]:
# Print the balanced_accuracy score of the model
print(balanced_accuracy_score(y_test, x_test_pred_2))

0.9961345712466498


In [60]:
# Generate a confusion matrix for the model
print(confusion_matrix(y_test, x_test_pred_2))

[[22397   114]
 [    2   748]]


In [61]:
# Print the classification report for the model
print(classification_report(y_test, x_test_pred_2))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     22511
           1       0.87      1.00      0.93       750

    accuracy                           1.00     23261
   macro avg       0.93      1.00      0.96     23261
weighted avg       1.00      1.00      1.00     23261



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** This is a much better prediction of both the healthy and high-risk loans. This prediction with oversampled data, predicts with 100% accuracy.